In [1]:
import os
import os.path
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
tf.config.list_physical_devices('GPU')

# Disable GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = ""



[]

In [3]:
from cmath import inf


x = []
y = []
cnt = 0
ind = []

for dirpath, dirnames, filenames in os.walk("../SisFall_dataset"):
    for filename in [f for f in filenames if f.endswith(".txt")]:
        if(filename != "Readme.txt"):
            
            if filename[0] == "D":
                # ADL file
                df = pd.read_csv(os.path.join(dirpath, filename),header=None)
                ax = np.array(df[0])*32/8192
                ay = np.array(df[1])*32/8192
                az = np.array(df[2])*32/8192
                aax = ax[np.arange(0,ax.shape[0],8)]
                aay = ay[np.arange(0,ax.shape[0],8)]
                aaz = az[np.arange(0,ax.shape[0],8)]
                s = np.sqrt(aax**2 + aay**2 + aaz**2)
                for i in range(s.shape[0]):
                    if s[i] >= 1.6:
                        st = i - 38
                        ed =  i + 39
                        if st < 0 or ed >= s.shape[0]:
                            continue
                        
                        tmp = np.array(list(zip(aax[st:ed],aay[st:ed],aaz[st:ed])))

                        
                        x.append(tmp)
                        y.append(1)


            else:
                # Fall Data
                df = pd.read_csv(os.path.join(dirpath, filename),header=None)
                ax = np.array(df[0])*32/8192
                ay = np.array(df[1])*32/8192
                az = np.array(df[2])*32/8192
                m = 0
                aax = ax[np.arange(m,ax.shape[0],8)]
                aay = ay[np.arange(m,ax.shape[0],8)]
                aaz = az[np.arange(m,ax.shape[0],8)]
                s = np.sqrt(aax**2 + aay**2 + aaz**2)

                i = np.argmax(s)
                cnt += 1
                st = i - 38
                ed =  i + 39

                if st < 0 or ed >= s.shape[0] or s[i] < 1.6:
                    continue

                tmp = np.array(list(zip(aax[st:ed],aay[st:ed],aaz[st:ed])))
                x.append(tmp)
                y.append(0)


In [4]:
print(cnt)

1798


In [5]:
X = np.array(x)
Y = np.array(y)
# x = X
# y = Y
print(X.shape,Y.shape)

(78723, 77, 3) (78723,)


In [6]:

X = X.reshape((-1, 77, 3, 1))
print(X.shape,Y.shape)
from collections import Counter
cnt =  Counter(Y)
cnt

(78723, 77, 3, 1) (78723,)


Counter({1: 76925, 0: 1798})

In [7]:
from sklearn.model_selection import train_test_split
X_tr, X_test, y_tr, y_test = train_test_split(X, Y, test_size=0.2, random_state=32)

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
orig_shape = X_tr.shape

wX = X_tr.reshape(-1,231)
print(X.shape,Y.shape,wX.shape)

over = SMOTE(sampling_strategy=1)
steps = [('o', over)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X_train, y_train = pipeline.fit_resample(wX, y_tr)

X_train = X_train.reshape((-1, 77, 3, 1))
y_test = keras.utils.to_categorical(y_test)
y_train = keras.utils.to_categorical(y_train)
print(X_train.shape,y_train.shape)


(78723, 77, 3, 1) (78723,) (62978, 231)
(123040, 77, 3, 1) (123040, 2)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten,Dropout
model = Sequential()
model.add(Conv2D(10, kernel_size=(5,3), activation='relu', input_shape=(77,3,1),strides=3))

model.add(Flatten())
model.add(Dense(2, activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 1, 10)         160       
                                                                 
 flatten (Flatten)           (None, 250)               0         
                                                                 
 dense (Dense)               (None, 2)                 502       
                                                                 
Total params: 662
Trainable params: 662
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20,batch_size=128)

Epoch 1/20
962/962 [==============================] - 3s 3ms/step - loss: 0.0560 - accuracy: 0.9844 - val_loss: 0.0104 - val_accuracy: 0.9982
Epoch 2/20
962/962 [==============================] - 3s 3ms/step - loss: 0.0114 - accuracy: 0.9971 - val_loss: 0.0078 - val_accuracy: 0.9985
Epoch 3/20
962/962 [==============================] - 3s 3ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.0073 - val_accuracy: 0.9985
Epoch 4/20
962/962 [==============================] - 3s 3ms/step - loss: 0.0038 - accuracy: 0.9989 - val_loss: 0.0052 - val_accuracy: 0.9987
Epoch 5/20
962/962 [==============================] - 2s 3ms/step - loss: 0.0026 - accuracy: 0.9993 - val_loss: 0.0050 - val_accuracy: 0.9987
Epoch 6/20
962/962 [==============================] - 2s 3ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0043 - val_accuracy: 0.9993
Epoch 7/20
962/962 [==============================] - 3s 4ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.0045 - val_accuracy: 0.9992
Epoch 

In [10]:
print(X_test.shape,y_test.shape)
results = model.evaluate(X_test, y_test, batch_size=128)

(15745, 77, 3, 1) (15745, 2)
124/124 [==============================] - 0s 2ms/step - loss: 0.0060 - accuracy: 0.9997


In [11]:
from sklearn.metrics import confusion_matrix


predictions = model.predict(X, batch_size=128, verbose=1)
classes_x=(np.argmax(predictions,axis=1))
true_classes = y

# y_test_flat = np.argmax(y_test, axis=-1)
c = confusion_matrix(y, classes_x)
print('Confusion matrix:\n', c)
print('Accuracy', (c[0, 0]+c[1, 1]) / (c[0, 1] + c[0, 0] + c[1, 1] + c[1, 0]))
print('sensitivity', c[0, 0] / (c[0, 1] + c[0, 0]))
print('specificity', c[1, 1] / (c[1, 1] + c[1, 0]))

616/616 [==============================] - 1s 1ms/step
Confusion matrix:
 [[ 1794     4]
 [    3 76922]]
Accuracy 0.9999110806244681
sensitivity 0.9977753058954394
specificity 0.9999610009749756


In [12]:

x_flattened = X_train.reshape(-1,231)
print(x_flattened.shape)
model2 = Sequential()
model2.add(Dense(64, activation='relu', input_dim=231))
model2.add(Dense(2, activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model2.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

(123040, 231)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                14848     
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 14,978
Trainable params: 14,978
Non-trainable params: 0
_________________________________________________________________


In [13]:

model2.fit(x_flattened, y_train, validation_split=0.25, epochs=20,batch_size=128)

Epoch 1/20
721/721 [==============================] - 2s 2ms/step - loss: 0.0179 - accuracy: 0.9943 - val_loss: 0.0068 - val_accuracy: 0.9983
Epoch 2/20
721/721 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9993 - val_loss: 0.0020 - val_accuracy: 0.9996
Epoch 3/20
721/721 [==============================] - 2s 2ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 4/20
721/721 [==============================] - 2s 2ms/step - loss: 4.4081e-04 - accuracy: 0.9999 - val_loss: 6.8960e-04 - val_accuracy: 0.9999
Epoch 5/20
721/721 [==============================] - 1s 2ms/step - loss: 2.9798e-04 - accuracy: 1.0000 - val_loss: 1.8819e-04 - val_accuracy: 1.0000
Epoch 6/20
721/721 [==============================] - 1s 2ms/step - loss: 1.5403e-04 - accuracy: 1.0000 - val_loss: 0.0051 - val_accuracy: 0.9979
Epoch 7/20
721/721 [==============================] - 2s 2ms/step - loss: 2.4798e-04 - accuracy: 1.0000 - val_loss: 3.9707e-04 -

In [14]:
print(X_test.shape,y_test.shape)
xt_flattened = X_test.reshape(-1,231)
results = model2.evaluate(xt_flattened, y_test, batch_size=128)


(15745, 77, 3, 1) (15745, 2)
124/124 [==============================] - 0s 1ms/step - loss: 0.0080 - accuracy: 0.9993


In [15]:
from sklearn.metrics import confusion_matrix


predictions = model2.predict(xt_flattened, batch_size=128, verbose=1)
classes_x=(np.argmax(predictions,axis=1))
true_classes = y_test

y_tt = np.argmax(y_test, axis=-1)
print(y_tt.shape,classes_x.shape)
c = confusion_matrix(y_tt, classes_x)
print('Confusion matrix:\n', c)
print('sensitivity', c[0, 0] / (c[0, 1] + c[0, 0]))
print('specificity', c[1, 1] / (c[1, 1] + c[1, 0]))

124/124 [==============================] - 0s 854us/step
(15745,) (15745,)
Confusion matrix:
 [[  333     7]
 [    4 15401]]
sensitivity 0.9794117647058823
specificity 0.9997403440441415
